<a href="https://colab.research.google.com/github/wlrma0108/Stand_Alone_Deeplearning/blob/main/Stand_Alone%234.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install torch

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import argparse
import numpy as np

In [4]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
trainset,valset=torch.utils.data.random_split(trainset,[40000,10000])
testset=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)

trainloader=torch.utils.data.DataLoader(trainset,batch_size=4,shuffle=True,num_workers=2)
valloader=torch.utils.data.DataLoader(valset,batch_size=4,shuffle=False)
testloader=torch.utils.data.DataLoader(testset,batch_size=4,shuffle=False,num_workers=2)

classes={'plane','car','bird','cat','deer','dog','frog','horse','ship','truck'}

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [12]:
class MLP(nn.Module):
  def __init__(self,in_dim,out_dim,hid_dim,n_layer,act):
    super(MLP,self).__init__()
    self.in_dim=in_dim
    self.out_dim=out_dim
    self.hid_dim=hid_dim
    self.n_layer=n_layer
    self.act=act

    self.fc=nn.Linear(self.in_dim,self.hid_dim)
    self.linears=nn.ModuleList()

    for i in range(self.n_layer-1):
      self.linears.append(nn.Linear(self.hid_dim,self.hid_dim))
    self.fc2 =nn.Linear(self.hid_dim,self.out_dim)

    if self.act=='relu':
      self.act=nn.ReLU()

  def forward(self, x):
      x=self.act(self.fc(x))
      for fc in self.linears:
        x=self.act(fc(x))
      x=self.fc2(x)
      return x
net=MLP(3072,10,100,4,'relu')

In [14]:
def experiment(args):
  
  net=MLP(args.in_dim,args.out_dim,args.hid_dim,args.n_layer,args.act)
  print(net)

  criterion=nn.CrossEntropyLoss()
  optimizer=optim.SGD(net.parameters(), lr=args.lr,momentum=args.mm)
  for epoch in range(args.epoch):

    net.train()
    running_loss=0.0
    train_loss=0.0
    for i , data in enumerate(trainloader, 0):
      optimizer.zero_grad()
      inputs,labels=data
      inputs=inputs.view(-1,3072)
      inputs=inputs.cuda()
      labels=labels.cuda()

      outputs=net(input)
      loss=criterion(outputs,labels)
      loss.backward()
      optimizer.step()

      running_loss+=loss.item()
      train_loss+=loss.item()

      if i%2000 ==1999:
        print('[%d,%5d loss:  %.3f'(epoch+1,i+1,running_loss/2000))
        running_loss=0.0
      
      net.eval()

      correct=0
      total=0
      val_loss=0
      with torch.no_grad():
        for data in valloader:
          images,labels=data
          images=images.view(-1,3072)
          images=images.cuda()
          labels=labels.cuda()

          outputs=net(images)
          loss = criterion(outputs.labels)
          val_loss+=loss.item()

          _,predicted=torch.max(outputs.data,1)
          total+=labels.size(0)
          correct+=(predicted==labels).sum().item()

        val_loss=val_loss/len(valloader)
        val_acc=100*correct/total
    print('Epoch {}, Train Loss: {}, Val Loss: {}, Val Acc: {}'.format(epoch, train_loss, val_loss, val_acc ))
  net.eval()
  correct=0
  total=0
  with torch.no_grad():
        for data in valloader:
          images,labels=data
          images=images.view(-1,3072)
          images=images.cuda()
          labels=labels.cuda()
          
          outputs=net(images)
          loss = criterion(outputs.labels)
          val_loss+=loss.item()

          _,predicted=torch.max(outputs.data,1)
          total+=labels.size(0)
          correct+=(predicted==labels).sum().item()
      
        test_acc=100*correct/total

  return train_loss,val_loss,val_acc,test_acc

In [15]:
seed=123
np.random.seed(seed)
torch.manual_seed(seed)
parser=argparse.ArgumentParser()
args=parser.parse_args("")

args.n_layer=5
args.in_dim=3072
args.out_dim=10
args.hid_dim=100
args.act='relu'

args.lr=0.0001
args.mm=0.9
args.epoch=2

list_val1=[4,5,6]
list_val2=[50,100,150]

for val1 in list_val1:
  for var2 in list_val2:
    args.n_layer =val1
    args.hid_dim=var2
    result =experiment(args)
    print(result)

MLP(
  (fc): Linear(in_features=3072, out_features=50, bias=True)
  (linears): ModuleList(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
  )
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (act): ReLU()
)
[1,  2000] loss: 2.305
[1,  4000] loss: 2.301
[1,  6000] loss: 2.295
[1,  8000] loss: 2.283
[1, 10000] loss: 2.251
Epoch 0, Train Loss: 22867.953108787537, Val Loss: 2.2226913457870485, Val Acc: 20.38
[2,  2000] loss: 2.178
[2,  4000] loss: 2.113
[2,  6000] loss: 2.048
[2,  8000] loss: 2.000
[2, 10000] loss: 1.942
Epoch 1, Train Loss: 20561.37340861559, Val Loss: 1.9249042528152467, Val Acc: 28.99
(20561.37340861559, 1.9249042528152467, 28.99, 29.2)
MLP(
  (fc): Linear(in_features=3072, out_features=100, bias=True)
  (linears): ModuleList(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): Linear(in_features=100, out_fe